In [2]:
#Import libraries
import os
import sys
import subprocess
import argparse
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, XSD, RDFS
from urllib.parse import quote


ModuleNotFoundError: No module named 'rdflib'

In [3]:
#read the finalized dataset

df_final = pd.read_csv("C:/Users/20204018/OneDrive - TU Eindhoven/Documents/Master DSAI/YEAR 1 Q4/Knowledge Engineering Project/Datasets/final_dataset.csv")
#check if the dataset is empty



FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/20204018/OneDrive - TU Eindhoven/Documents/Master DSAI/YEAR 1 Q4/Knowledge Engineering Project/Datasets/final_dataset.csv'

In [ ]:
# Define the mobility score function
def compute_mobility_score(row):
    return round((
        0.4 * (10 / (row['afstand_tot_treinstation'] + 1)) +
        0.15 * (10 / (row['afstand_tot_ziekenhuis'] + 1)) +
        0.10 * row['ziekenhuizen_binnen_10km'] +
        0.10 * (10 / (row['afstand_tot_oprit_snelweg'] + 1)) +
        0.10 * (10 / (row['afstand_tot_belangrijk_knooppunt'] + 1)) +
        0.15 * (
            row['supermarkten_binnen_1km'] +
            row['basisschool_binnen_1km'] +
            row['kinderopvang_binnen_1km'] +
            row['restaurants_binnen_1km'] +
            row['huisartsenpraktijken_binnen_1km']
        )
    ), 3)

# Apply the function
df_final['mobility_score'] = df_final.apply(compute_mobility_score, axis=1)

In [ ]:
#We use density to essential services within 1km 

,gemeente,aantal_geregistreerde_personenautos_per_inwoner,aantal_geregistreerde_personenautos_per_huishouden,aantal_geregistreerde_personenautos_per_woning,bevolking_totaal_aantal,huishoudens_aantal,gemiddeld_inkomen_per_inwoner,gemiddeld_inkomen_per_huishouden,woningdichtheid_aantal_woningen_per_hectare,provincie,...,basisschool_binnen_3km,basisschool_binnen_5km,afstand_tot_voortgezet_onderwijs,voortgezet_onderwijs_binnen_3km,voortgezet_onderwijs_binnen_5km,voortgezet_onderwijs_binnen_10km,afstand_tot_oprit_snelweg,afstand_tot_treinstation,afstand_tot_belangrijk_knooppunt,mobility_score
42,Borger-Odoorn,0.60,1.36,1.37,25919,11462,30100.0,68100.0,0.42,Drenthe,...,2.2,4.8,5.6,0.4,1.1,2.7,2.0,15.5,16.6,1.367
20,Baarle-Nassau,0.61,1.38,1.47,7071,3140,31400.0,70700.0,0.39,Noord-Brabant,...,0.8,1.1,2.5,0.6,0.8,1.0,2.1,19.4,19.8,1.421
248,Schiermonnikoog,0.37,0.63,0.60,982,581,28650.0,63150.0,0.14,Fryslân,...,1.0,1.0,0.4,1.0,1.0,1.0,15.5,39.0,57.1,1.463
316,Westerwolde,0.57,1.33,1.25,26537,11346,27900.0,65300.0,0.44,Groningen,...,2.6,4.0,4.2,0.5,1.0,2.3,1.2,14.7,29.3,1.516
28,Bergeijk,0.59,1.37,1.37,19092,8196,33400.0,77800.0,0.81,Noord-Brabant,...,3.5,6.8,7.1,0.0,0.0,1.5,3.4,19.3,20.1,1.544
143,Hulst,0.59,1.27,1.19,27596,12836,32500.0,69900.0,0.68,Zeeland,...,2.5,4.5,4.8,1.2,1.9,2.5,2.0,46.9,50.7,1.560
217,Opsterland,0.56,1.33,1.27,30054,12780,29600.0,69600.0,0.59,Fryslân,...,2.2,4.1,4.7,0.3,0.6,4.0,1.9,16.9,19.1,1.568
314,Westerveld,0.60,1.33,1.35,19860,8892,32000.0,71500.0,0.31,Drenthe,...,1.2,2.7,6.6,0.2,0.2,2.2,1.1,11.9,19.5,1.597
214,Ooststellingwerf,0.57,1.28,1.29,25837,11546,28600.0,64000.0,0.51,Fryslân,...,2.6,3.4,4.6,0.4,0.6,1.4,1.5,22.2,23.8,1.615
11,Altena,0.54,1.32,1.33,57726,23447,31400.0,77300.0,1.16,Noord-Brabant,...,2.8,4.9,3.9,0.5,0.9,3.9,2.0,12.5,22.2,1.624


In [21]:
from rdflib import Graph, Namespace, Literal, RDF, RDFS, XSD, BNode
from urllib.parse import quote

# Namespaces
MOB = Namespace("http://example.org/mobility#")
EX = Namespace("http://example.org/resource/")
g = Graph()
g.bind("mob", MOB)
g.bind("ex", EX)
g.bind("rdfs", RDFS)

# Define facility types globally for labels
facility_types = [
    "TrainStation", "Hospital", "Supermarket", "Restaurant", "Childcare",
    "PrimarySchool", "SecondarySchool", "Highway", "Hub", "GP"
]
for facility in facility_types:
    facility_uri = MOB[facility]
    g.add((facility_uri, RDF.type, MOB.FacilityType))
    g.add((facility_uri, RDFS.label, Literal(facility.replace("_", " "))))

for _, row in df_final.iterrows():
    code = quote(row['gemeente_code'].strip())
    gemeente_uri = EX[f"gemeente/{code}"]
    province = quote(row['provincie'].strip().replace(" ", "_"))
    province_uri = EX[f"provincie/{province}"]

    # Province
    g.add((province_uri, RDF.type, MOB.Province))
    g.add((province_uri, MOB.name, Literal(row['provincie'])))
    g.add((province_uri, RDFS.label, Literal(row['provincie'])))

    # Municipality
    g.add((gemeente_uri, RDF.type, MOB.Municipality))
    g.add((gemeente_uri, MOB.name, Literal(row['gemeente'])))
    g.add((gemeente_uri, MOB.isIn, province_uri))
    g.add((gemeente_uri, RDFS.label, Literal(row['gemeente'])))

    # Mobility Profile
    profile_uri = EX[f"mobilityProfile/{code}"]
    g.add((profile_uri, RDF.type, MOB.MobilityProfile))
    g.add((profile_uri, MOB.mobilityScore, Literal(row['mobility_score'], datatype=XSD.float)))
    g.add((profile_uri, MOB.population, Literal(row['bevolking_totaal_aantal'], datatype=XSD.integer)))
    g.add((profile_uri, MOB.carOwnershipPerHousehold, Literal(row['aantal_geregistreerde_personenautos_per_huishouden'], datatype=XSD.float)))
    g.add((profile_uri, MOB.carOwnershipPerResident, Literal(row['aantal_geregistreerde_personenautos_per_inwoner'], datatype=XSD.float)))
    g.add((gemeente_uri, MOB.hasMobilityProfile, profile_uri))
    g.add((profile_uri, RDFS.label, Literal(f"Mobility profile of {row['gemeente']}")))

    # Access Facilities with URI instead of BNode
    facilities = {
        "TrainStation": row['afstand_tot_treinstation'],
        "Hospital": row['afstand_tot_ziekenhuis'],
        "Supermarket": row['afstand_tot_supermarkt'],
        "Restaurant": row['afstand_tot_restaurant'],
        "Childcare": row['afstand_tot_kinderopvang'],
        "PrimarySchool": row['afstand_tot_basisschool'],
        "SecondarySchool": row['afstand_tot_voortgezet_onderwijs'],
        "Highway": row['afstand_tot_oprit_snelweg'],
        "Hub": row['afstand_tot_belangrijk_knooppunt'],
        "GP": row['afstand_tot_huisartsenpraktijk']
    }

    for facility, distance in facilities.items():
        access_uri = EX[f"access/{code}/{facility.lower()}"]
        g.add((access_uri, RDF.type, MOB.Access))
        g.add((access_uri, MOB.inMunicipality, gemeente_uri))
        g.add((access_uri, MOB.facilityType, MOB[facility]))
        g.add((access_uri, MOB.distanceKm, Literal(distance, datatype=XSD.float)))
        g.add((access_uri, RDFS.label, Literal(f"{facility} access in {row['gemeente']}")))

    # Facility Density as direct properties on MobilityProfile
    densities = {
        "supermarketCountWithin1km": row['supermarkten_binnen_1km'],
        "hospitalDensity10km": row['ziekenhuizen_binnen_10km'],
        "primarySchoolCountWithin1km": row['basisschool_binnen_1km'],
        "restaurantCountWithin1km": row['restaurants_binnen_1km'],
        "gpCountWithin1km": row['huisartsenpraktijken_binnen_1km']
    }

    for prop, value in densities.items():
        g.add((profile_uri, MOB[prop], Literal(value, datatype=XSD.float)))

# Save to file
g.serialize(destination="mobility_graph.ttl", format="turtle")
print("✔ RDF Turtle file generated: mobility_graph.ttl (refined)")


✔ RDF Turtle file generated: mobility_graph.ttl (refined)
